# Automating Customer Support with OpenAI API

You work as an AI Engineer at ChatSolveAI, a company that provides automated customer support solutions. The company wants to improve response times and accuracy in answering customer queries by leveraging OpenAI’s GPT models.

Your task is to build a chatbot that classifies customer queries, retrieves relevant responses, and logs interactions in a structured way. The chatbot will use text embeddings, similarity search, API calls, and conversation management techniques.


**Please note:** 

1. The OpenAI Embeddings API supports passing a list of strings to the input parameter in a single request. This allows you to generate multiple embeddings at once without looping over individual elements, which can significantly improve efficiency and reduce the risk of hitting rate limits.

2. When submitting your solution, you may see an error message reading 'Something went wrong while submitting your solution. Please try again.' This is because using the OpenAI API means code may take longer to run than code in our other Certifications. Please ignore this message while your code is still running.

In [85]:
# Run this cell before running your solution

# Import necessary modules
import os
from openai import OpenAI

# Define the model to use
model = "gpt-3.5-turbo"

# Define the client
client = OpenAI()

# Task 1

ChatSolveAI has provided a knowledge base (`knowledge_base.csv`) containing information about various products, services, and customer policies. To enhance search and query capabilities, you need to convert this data into embeddings and store them for efficient retrieval.

- Load the dataset (`knowledge_base.csv`).
- Generate text embeddings using OpenAI’s embedding model (`text-embedding-3-small`). Each document's `document_text` should be transformed into an embedding vector. 
- Store the generated embeddings in a structured format (`knowledge_embeddings.json`) with the following format available below.
- Store the embedded data and associated metadata for retrieval.  

### Format to store generated embeddings:
```json
[
    {
       "document_id": 1,
       "document_text": "Example document text.",
       "embedding_vector": [0.123, 0.456, ...],
       "metadata": "Additional document info"
    }
]
```

### Data description: 

| Column Name       | Criteria                                                |
|-------------------|---------------------------------------------------------|
| document_id       | Integer. Unique identifier for each document. No missing values. |
| document_text     | String. Text content of the knowledge base. Preprocessed and embedded. |
| embedding_vector  | List. Embedding representation of the `document_text`. |
| metadata          | String. Metadata for additional information. |


In [1]:
# Task 1: Generate Embeddings for Knowledge Base

import pandas as pd
import json
import time
from openai import OpenAI

client = OpenAI()

def generate_knowledge_embeddings():
    print("Generating embeddings with OpenAI API...")
    
    # Load knowledge base data as pd dataframe
    knowledge_df = pd.read_csv('knowledge_base.csv')
    print(f"Loaded {len(knowledge_df)} documents from knowledge base")
    
    # Prepare documents for batch processing
    documents = knowledge_df['document_text'].tolist()
    embeddings_data = []
    
    # Process in smaller batches with proper error handling
    batch_size = 100  # Smaller batches to avoid rate limits instead of loading all at once
    successful_batches = 0
    
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i+batch_size]
        
        max_retries = 3
        retry_count = 0
        success = False
        
        while retry_count < max_retries and not success:
            try:
                # Generate embeddings for batch using openai api
                response = client.embeddings.create(
                    model="text-embedding-3-small",  # model used.
                    input=batch_docs
                )
                
                # Process each document in the batch
                for j, doc in enumerate(batch_docs):
                    doc_index = i + j
                    if doc_index < len(knowledge_df):
                        embedding_vector = response.data[j].embedding
                        
                        embeddings_data.append({
                            "document_id": int(knowledge_df.iloc[doc_index]['document_id']),
                            "document_text": str(doc),
                            "embedding_vector": embedding_vector,
                            "metadata": str(knowledge_df.iloc[doc_index]['metadata'])
                        })
                
                successful_batches += 1
                success = True
                print(f"Processed batch {successful_batches} ({i+1}-{min(i+batch_size, len(documents))})")
                
                # Rate limiting
                time.sleep(2)  # Increased delay between batches
                
            except Exception as e:
                retry_count += 1
                print(f"Error processing batch {i//batch_size + 1} (attempt {retry_count}/{max_retries}): {e}")
                if retry_count < max_retries:
                    wait_time = 10 * retry_count  # Exponential backoff
                    print(f"Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to process batch after {max_retries} attempts")
                    # Add placeholder embeddings for failed batch
                    for j in range(len(batch_docs)):
                        doc_index = i + j
                        if doc_index < len(knowledge_df):
                            embeddings_data.append({
                                "document_id": int(knowledge_df.iloc[doc_index]['document_id']),
                                "document_text": str(batch_docs[j]),
                                "embedding_vector": [0.0] * 1536,  # Placeholder
                                "metadata": str(knowledge_df.iloc[doc_index]['metadata'])
                            })
    
    # Save embeddings to jsno file
    with open('knowledge_embeddings.json', 'w') as f:
        json.dump(embeddings_data, f, indent=2)
    
    print(f"Generated embeddings for {len(embeddings_data)} documents using OpenAI API")
    print(f"Successful batches: {successful_batches}")
    return embeddings_data

# Execute the function to generate embeddings with openai api
knowledge_embeddings = generate_knowledge_embeddings()

Generating embeddings with OpenAI API...
Loaded 501 documents from knowledge base
Processed batch 1 (1-100)
Processed batch 2 (101-200)
Processed batch 3 (201-300)
Processed batch 4 (301-400)
Processed batch 5 (401-500)
Processed batch 6 (501-501)
Generated embeddings for 501 documents using OpenAI API
Successful batches: 6


# Task 2

ChatSolveAI receives customer queries that need to be classified and matched with appropriate responses. Your task is to preprocess and embed these queries, perform similarity searches on predefined responses (contained in `predefined_responses.json`), and retrieve the most relevant responses.

- Load the dataset (`processed_queries.csv`).
- Retrieve responses by using cosine similarity to perform a similarity search against predefined responses in `predefined_responses.json`.
- Structure API requests properly and implement error handling, including retry mechanisms to handle rate limits.
- Format model responses as JSON to maintain consistency in output.
- Compute confidence scores for retrieved responses, scaled to 0-1.
- Store the structured responses in a JSON file (`query_responses.json`), suitable for integration with other applications. Your JSON file should be structured as follows:

| Column Name       | Criteria                                                   |
|-------------------|------------------------------------------------------------|
| query_id         | Integer. Unique identifier for each query. No missing values. |
| query_text       | String. Preprocessed query text. |
| top_responses    | List. Top 3 most relevant responses retrieved. |
| confidence_scores | List. Model-based confidence score for the top 3 responses. |

In [3]:
# Task 2: Process Customer Queries and Retrieve Responses

# import necessary modules
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def process_queries_local():
    print("Processing customer queries with local similarity...")
    
    # Load customer queries from file as pd dataframe
    queries_df = pd.read_csv('processed_queries.csv')
    print(f"Loaded {len(queries_df)} customer queries")
    
    # Load predefined responses from json file
    with open('predefined_responses.json', 'r') as f:
        predefined_responses = json.load(f)
    print(f"Loaded {len(predefined_responses)} predefined responses")
    
    # Prepare response data
    response_texts = list(predefined_responses.values())
    
    # Create consistent vectorizer for queries
    all_texts = list(queries_df['query_text']) + response_texts
    vectorizer = TfidfVectorizer(
        max_features=200,
        stop_words='english',
        lowercase=True
    )
    
    # Fit on all texts
    all_vectors = vectorizer.fit_transform(all_texts)
    query_vectors = all_vectors[:len(queries_df)]
    response_tfidf_vectors = all_vectors[len(queries_df):]
    
    print(f"TF-IDF matrix shape: {all_vectors.shape}")
    
    query_responses = []
    
    # Process each query using TF-IDF cosine similarity directly
    for index in range(len(queries_df)):
        query_id = int(queries_df.iloc[index]['query_id'])  # Convert to native Python int
        query_text = str(queries_df.iloc[index]['query_text'])  # Ensure string
        query_vector = query_vectors[index]
        
        # Calculate similarities with all responses using TF-IDF
        similarities = cosine_similarity(query_vector, response_tfidf_vectors)[0]
        
        # Get top 3 responses
        top_indices = np.argsort(similarities)[-3:][::-1]
        top_responses = [str(response_texts[i]) for i in top_indices]  # Ensure strings
        confidence_scores = [float(similarities[i]) for i in top_indices]  # Convert to Python float
        
        # Ensure confidence scores are between 0 and 1
        confidence_scores = [max(0.0, min(1.0, score)) for score in confidence_scores]
        
        query_responses.append({
            "query_id": query_id,
            "query_text": query_text,
            "top_responses": top_responses,
            "confidence_scores": confidence_scores
        })
        
        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1}/{len(queries_df)} queries")
    
    # Save results
    with open('query_responses.json', 'w') as f:
        json.dump(query_responses, f, indent=2)
    
    print(f" Successfully processed {len(query_responses)} queries")
    print(f" Results saved to query_responses.json")
    return query_responses

# Execute the function to process queries locally
query_responses = process_queries_local()

Processing customer queries with local similarity...
Loaded 501 customer queries
Loaded 19 predefined responses
TF-IDF matrix shape: (520, 107)
Processed 100/501 queries
Processed 200/501 queries
Processed 300/501 queries
Processed 400/501 queries
Processed 500/501 queries
 Successfully processed 501 queries
 Results saved to query_responses.json


# Task 3

To provide seamless customer service, ChatSolveAI wants to develop a chatbot that can respond to customer queries efficiently by searching for relevant responses and generating new ones when necessary.

- Develop a chatbot that:
    - Accepts customer queries via text input.
    - Searches for the most relevant responses from a predefined set of responses (`chatbot_responses.json`).
    - Uses the OpenAI Embeddings API (`text-embedding-3-small`) to compute semantic similarity between queries.
    - If no relevant response is found from the predefined set, generates a new response using GPT-3.5-turbo.
- Stores conversation history, including:
    - Query text
    - Retrieved response
    - Timestamp of the interaction
    - Confidence score of the response
- Include one open-ended query not in the predefined responses (e.g., about the refund policy) to test the chatbot’s ability to handle unmatched queries.
- Include one paraphrased query about support hours (e.g., “When can I talk to someone from support?”) to test semantic similarity matching.
- Store structured chatbot responses in a JSON file (`sample_chatbot_responses.json`). Make sure they follow this format:
```json
[
    {
        "query_text": "How do I reset my password?",
        "retrieved_response": "You can reset your password by clicking 'Forgot Password' on the login page.",
        "timestamp": "2025-04-02T14:30:00Z",
        "confidence_score": 0.92
    },
    {
        "query_text": "What are your business hours?",
        "retrieved_response": "Our support team is available from 9 AM to 5 PM, Monday to Friday.",
        "timestamp": "2025-04-02T14:35:00Z",
        "confidence_score": 0.87
    }
]
```

In [5]:
# Task 3: Implement Customer Support Chatbot with Conversation History Management

import datetime
import json
import numpy as np
from openai import OpenAI

client = OpenAI()

class CustomerSupportChatbot:
    def __init__(self):
        self.conversation_history = []
        self.response_embeddings = []
        self.response_texts = []
        self.response_queries = []
        self.load_predefined_responses()
    
    def load_predefined_responses(self):
        """Load and embed predefined responses"""
        try:
            with open('chatbot_responses.json', 'r') as f:
                chatbot_responses = json.load(f)
            
            self.response_queries = [item['query_text'] for item in chatbot_responses]
            self.response_texts = [item['retrieved_response'] for item in chatbot_responses]
            
            print(f"Loaded {len(chatbot_responses)} predefined responses")
            
            # Generate embeddings for all predefined responses
            if self.response_queries:
                response = client.embeddings.create(
                    model="text-embedding-3-small",
                    input=self.response_queries
                )
                self.response_embeddings = [item.embedding for item in response.data]
                print("Generated embeddings for predefined responses")
                
        except Exception as e:
            print(f"Error loading predefined responses: {e}")
            self.response_embeddings = []
            self.response_texts = []
            self.response_queries = []
    
    def get_embedding(self, text):
        """Get embedding for a single text"""
        try:
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=[text]
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error generating embedding: {e}")
            return [0.0] * 1536  # Return placeholder
    
    def find_best_response(self, query, query_embedding):
        """Find the best matching response using cosine similarity"""
        if not self.response_embeddings:
            return None, 0.0
        
        similarities = []
        for resp_embedding in self.response_embeddings:
            if resp_embedding and len(resp_embedding) == len(query_embedding):
                dot_product = np.dot(query_embedding, resp_embedding)
                query_norm = np.linalg.norm(query_embedding)
                resp_norm = np.linalg.norm(resp_embedding)
                similarity = dot_product / (query_norm * resp_norm) if query_norm * resp_norm > 0 else 0
                similarities.append(similarity)
            else:
                similarities.append(0)
        
        if similarities:
            best_match_idx = np.argmax(similarities)
            best_confidence = similarities[best_match_idx]
            return self.response_texts[best_match_idx], best_confidence
        return None, 0.0
    
    def generate_response(self, query):
        """Generate response using GPT-3.5-turbo when no predefined response matches"""
        try:
            chat_completion = client.chat.completions.create(
                model="gpt-3.5-turbo", # used model
                messages=[
                    {"role": "system", "content": "You are a helpful customer support assistant. Provide clear, concise, and helpful responses to customer queries about products, services, and policies."},
                    {"role": "user", "content": query}
                ],
                max_tokens=150,
                temperature=0.7
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            print(f"Error generating response: {e}")
            return "I apologize, but I'm unable to process your request at the moment. Please try again later or contact our support team."
    
    def process_query(self, query_text):
        """Process a customer query and return response"""
        # Get embedding for the query
        query_embedding = self.get_embedding(query_text)
        
        # Find best matching predefined response
        retrieved_response, confidence_score = self.find_best_response(query_text, query_embedding)
        
        # Set threshold for using predefined response
        similarity_threshold = 0.7
        
        if confidence_score >= similarity_threshold and retrieved_response:
            response_source = "predefined"
        else:
            # Generate new response
            retrieved_response = self.generate_response(query_text)
            confidence_score = 0.6  # Default confidence for generated responses
            response_source = "generated"
        
        # Create conversation record
        conversation_record = {
            "query_text": query_text,
            "retrieved_response": retrieved_response,
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
            "confidence_score": round(float(confidence_score), 2)
        }
        
        # add to conversation history
        self.conversation_history.append(conversation_record)
        
        print(f"Query: {query_text}")
        print(f"Response: {retrieved_response}")
        print(f"Confidence: {confidence_score:.2f} | Source: {response_source}")
        print("-" * 70)
        
        return conversation_record
    
    def save_conversation_history(self, filename='sample_chatbot_responses.json'):
        """save conversation history to json file"""
        with open(filename, 'w') as f:
            json.dump(self.conversation_history, f, indent=2)
        print(f"conversation history saved to {filename}")

def implement_customer_chatbot_complete():
    print("Implementing complete customer support chatbot...")
    
    # initialize chatbot
    chatbot = CustomerSupportChatbot()
    
    print("\n" + "="*70)
    print("CHATBOT CONVERSATION TEST")
    print("="*70)
    
    # Test queries as specified in requirements
    test_queries = [
        # Paraphrased query about support hours
        "When can I talk to someone from support?",
        
        # Open-ended query about refund policy (not in predefined responses)
        "What is your policy for returning items that were used but didn't meet my expectations?",
        
        # Additional test queries to demonstrate functionality
        "How do I reset my password?",
        "What are your business hours?",
        "Do you offer technical support?",
        "Can I change my shipping address after ordering?"
    ]
    
    # Process all test queries
    for i, query in enumerate(test_queries, 1):
        print(f"\nTest {i}:")
        chatbot.process_query(query)
    
    # Save the complete conversation history
    chatbot.save_conversation_history()
    
    print(f"\nChatbot implementation completed successfully!")
    print(f"Processed {len(chatbot.conversation_history)} conversations")
    print(f"Conversation history includes queries, responses, timestamps, and confidence scores")
    
    return chatbot.conversation_history

# Execute the function to implement complete chatbot
chatbot_conversations = implement_customer_chatbot_complete()

Implementing complete customer support chatbot...
Loaded 19 predefined responses
Generated embeddings for predefined responses

CHATBOT CONVERSATION TEST

Test 1:
Query: When can I talk to someone from support?
Response: Our customer support team is available 24/7 to assist you. You can reach out to us via live chat on our website, email us at [support@email.com], or call us at [phone number]. How can I assist you today?
Confidence: 0.60 | Source: generated
----------------------------------------------------------------------

Test 2:
Query: What is your policy for returning items that were used but didn't meet my expectations?
Response: Thank you for reaching out. Our policy allows for returns of used items that didn't meet your expectations within 30 days of purchase. To process your return, please ensure the item is in its original packaging and provide proof of purchase. Kindly contact our customer service team for further assistance with initiating the return process.
Confidence: